# How to read parquet file by connecting to s3 bucket and reading parquet file. 

## With pyathena

In [1]:
! pip install PyAthena

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [2]:
import pandas as pd
from pyathena import connect
from pyathena.pandas.cursor import PandasCursor
from pyathena.pandas.util import as_pandas

In [3]:
from datetime import datetime

In [4]:
print(datetime.now())

2021-08-25 12:25:30.099921


In [12]:
t1 = datetime.now()
cursor = connect(s3_staging_dir='s3://aquila-dev-bucket/aquila_sagemaker_metadata/',
                 region_name='us-east-1',
                 cursor_class=PandasCursor).cursor()
#df = cursor.execute("SELECT * FROM many_rows").as_pandas()
print("Total time taken for connection : {}".format(datetime.now() - t1))
t2 = datetime.now()
df = as_pandas(cursor.execute("SELECT * FROM awsdatacatalog.telemetry_data.phoenix_logs_parquet where year = '2020' and month ='02' and day ='08' LIMIT 100"))
print("Total time taken for querying data : {}".format(datetime.now() - t2))
#df = as_pandas(cursor)
#print(df.describe())


Total time taken for connection : 0:00:00.063139
Total time taken for querying data : 0:00:02.874443


In [17]:
#print(df.describe())
#print(df.head())
print(cursor.state)
print(cursor.state_change_reason)
print(cursor.completion_date_time)
print(cursor.submission_date_time)
print(cursor.data_scanned_in_bytes)
print(cursor.total_execution_time_in_millis)
print(cursor.output_location)
#print(df.describe())

SUCCEEDED
None
2021-08-25 12:28:25.007000+00:00
2021-08-25 12:28:23.777000+00:00
3353056
1230
s3://aquila-dev-bucket/aquila_sagemaker_metadata/a8f86caa-1f84-4454-92f8-9f4eed674aec.csv


In [13]:
df

,ts,service,type,tbl,job_status,attempt_no,workload,job_session_target_id,job_stime,resource_id,...,vm_error_msg,is_ebs_snapshot_exist,job_error_msg,vm_duration,job_error_code,clientid_2648281206,tablename,year,month,day
0,1581174004851564032,PhoenixStorageNodeWorker,telemetry,NaN,NaN,<NA>,NaN,<NA>,<NA>,<NA>,...,NaN,None,NaN,<NA>,<NA>,<NA>,NaN,2020,02,08
1,1581192118771632896,PhoenixStorageNodeWorker,telemetry,NaN,NaN,<NA>,NaN,<NA>,<NA>,<NA>,...,NaN,None,NaN,<NA>,<NA>,<NA>,NaN,2020,02,08
2,1581174004993391104,PhoenixStorageNodeWorker,telemetry,NaN,NaN,<NA>,NaN,<NA>,<NA>,<NA>,...,NaN,None,NaN,<NA>,<NA>,<NA>,NaN,2020,02,08
3,1581192123822118144,PhoenixStorageNodeWorker,telemetry,NaN,NaN,<NA>,NaN,<NA>,<NA>,<NA>,...,NaN,None,NaN,<NA>,<NA>,<NA>,NaN,2020,02,08
4,1581174005603941888,PhoenixStorageNodeWorker,telemetry,NaN,NaN,<NA>,NaN,<NA>,<NA>,<NA>,...,NaN,None,NaN,<NA>,<NA>,<NA>,NaN,2020,02,08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1581192200244784128,PhoenixStorageNodeWorker,telemetry,NaN,NaN,<NA>,NaN,<NA>,<NA>,<NA>,...,NaN,None,NaN,<NA>,<NA>,<NA>,NaN,2020,02,08
96,1581174048733299968,PhoenixStorageNode,telemetry,NaN,NaN,<NA>,NaN,<NA>,<NA>,<NA>,...,NaN,None,NaN,<NA>,<NA>,<NA>,NaN,2020,02,08
97,1581192200771684864,PhoenixVMwareConfig,telemetry,NaN,NaN,0,NaN,<NA>,<NA>,<NA>,...,NaN,None,NaN,<NA>,<NA>,<NA>,NaN,2020,02,08
98,1581174050080893952,PhoenixStorageNode,telemetry,NaN,NaN,<NA>,NaN,<NA>,<NA>,<NA>,...,NaN,None,NaN,<NA>,<NA>,<NA>,NaN,2020,02,08


In [2]:
!pip install awswrangler

     |████████████████████████████████| 184 kB 28.9 MB/s eta 0:00:01
     |████████████████████████████████| 94 kB 642 kB/s  eta 0:00:01
     |████████████████████████████████| 43 kB 168 kB/s  eta 0:00:01
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [3]:
import awswrangler as wr


In [9]:
%%time
df = wr.athena.read_sql_query(
    sql="SELECT * FROM {} where year = '2020' and month ='02' and day ='08' LIMIT 100".format("awsdatacatalog.telemetry_data.phoenix_logs_parquet"), database="telemetry_data"
)

CPU times: user 680 ms, sys: 8.56 ms, total: 689 ms
Wall time: 5.1 s


In [10]:
df.head(2)

,ts,service,type,tbl,job_status,attempt_no,workload,job_session_target_id,job_stime,resource_id,...,vm_error_msg,is_ebs_snapshot_exist,job_error_msg,vm_duration,job_error_code,clientid_2648281206,tablename,year,month,day
0,1581127710497745152,PhoenixStorageNodeWorker,telemetry,<NA>,<NA>,<NA>,SQL,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,2020,02,08
1,1581135151810349056,PhoenixStorageNodeWorker,telemetry,<NA>,<NA>,<NA>,SQL,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,2020,02,08
